<a href="https://colab.research.google.com/github/GustaveRw/Peti-Bot/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chatterbot google-api-python-client


In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

# Replace the API key below with a valid API key
api_key = "YOUR_API_KEY"

# Set up the Google Translate API service
service = build('translate', 'v2', credentials=Credentials.from_authorized_user_info(info={"api_key": api_key}))

# Create a new chatbot
chatbot = ChatBot("Peti")

# Train the chatbot with some pre-defined conversation
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english.greetings", "chatterbot.corpus.english.conversations")

# Define a function that will translate the chatbot's response
def translate_response(response, target_language):
    # Call the Google Translate API to translate the response
    translation = service.translations().list(source='en', target=target_language, q=[response]).execute()

    # Return the translated response
    return translation['translations'][0]['translatedText']

# Set the default language for the chatbot
default_language = "en"

# Start a conversation with the chatbot
while True:
    # Get the user's input
    request = input("You: ")

    # Determine the language of the user's input
    language = service.detections().list(q=[request]).execute()
    language = language['detections'][0][0]['language']

    # If the language is not the default language, translate the user's input into the default language
    if language != default_language:
        request = service.translations().list(source=language, target=default_language, q=[request]).execute()
        request = request['translations'][0]['translatedText']

    # Get a response from the chatbot
    response = chatbot.get_response(request).text

    # Translate the response into the user's language
    response = translate_response(response, language)

    # Print the response
    print("Chatbot: " + response)


In [ ]:
!pip install flask


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/', methods=['POST'])
def chatbot():
    # Get the user's input from the request
    request_json = request.get_json(force=True)
    request_text = request_json['text']

    # Process the user's input and get a response from the chatbot
    response_text = chatbot.get_response(request_text).text

    # Return the chatbot's response as JSON
    return jsonify({"text": response_text})

if __name__ == '__main__':
    app.run()


In [ ]:
!python app.py


In [ ]:
!ngrok http 5000


In [ ]:
!npx create-react-app

In [ ]:
!npm install axios


In [ ]:
import React, { useState } from 'react';
import axios from 'axios';


async function getChatbotResponse(message) {
  try {
    const response = await axios.post('http://localhost:5000/', {
      message: message
    });
    return response.data;
  } catch (error) {
    console.error(error);
  }
}


function Chatbot() {
  const [message, setMessage] = useState('');
  const [response, setResponse] = useState('');

  async function handleSubmit(event) {
    event.preventDefault();
    const chatbotResponse = await getChatbotResponse(message);
    setResponse(chatbotResponse);
  }

  return (
    <div className="flex flex-col items-center mt-10">
      <form className="w-full max-w-sm" onSubmit={handleSubmit}>
        <div className="flex items-center border-b border-teal-500 py-2">
          <input
            className="appearance-none bg-transparent border-none w-full text-gray-700 mr-3 py-1 px-2 leading-tight focus:outline-none"
            type="text"
            placeholder="Enter your message"
            value={message}
            onChange={event => setMessage(event.target.value)}
          />
          <button
            className="flex-shrink-0 bg-teal-500 hover:bg-teal-700 border-teal-500 hover:border-teal-700 text-sm border-4 text-white py-1 px-2 rounded"
            type="submit"
          >
            Send
          </button>
        </div>
      </form>
      <div className="w-full max-w-sm mt-10 text-center text-gray-700">
        {response}
      </div>
    </div>
  );
}

export default Chatbot;
